In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import helper
import librosa

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.regularizers import l2

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# set the random seed
#tf.random.set_seed(42)
#np.random.seed(42)



2.7.0
Num GPUs Available:  1


2022-02-09 23:44:50.265251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 23:44:50.303838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 23:44:50.304092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [17]:
dataset_root_dir = os.path.join("/home/charlesedwards/Documents", 'kaggle_2018_dataset')
data_npy_folder = os.path.join(dataset_root_dir, 'data')

# load the training data
dataset_train_folder = os.path.join(dataset_root_dir, 'train')
dataset_train_csv = os.path.join(dataset_train_folder, 'catalog.csv')

# load in our data_frame
metadata = pd.read_csv(dataset_train_csv)

model_dir = os.path.join( dataset_root_dir, 'models')
model_dir = os.path.join( model_dir, 'CC trained')
model_path = os.path.join(model_dir, 'K2018_MFCC_RESNET32_lr-5e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_categorical_crossentropy.h5')
#model_path = os.path.join(model_dir, 'K2018_MFCC_RESNET32_lr-1e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_categorical_crossentropy_optimized_train_full.h5')

# Pre-processed MFCC coefficients
X = np.load( os.path.join(data_npy_folder, "X-mfcc.npy" ) )
y = np.load( os.path.join(data_npy_folder, "y-mfcc.npy" ) )

labels = [ row["label"] for index, row in metadata.iterrows() ]

model_path, labels

('/home/charlesedwards/Documents/kaggle_2018_dataset/models/CC trained/K2018_MFCC_RESNET32_lr-5e-06_b1-0.99_b2-0.999_EPOCH-500_BATCH-32_categorical_crossentropy.h5',
 ['Hi-hat',
  'Saxophone',
  'Trumpet',
  'Glockenspiel',
  'Cello',
  'Cello',
  'Knock',
  'Gunshot_or_gunfire',
  'Clarinet',
  'Computer_keyboard',
  'Hi-hat',
  'Keys_jangling',
  'Snare_drum',
  'Writing',
  'Cello',
  'Laughter',
  'Laughter',
  'Tearing',
  'Fart',
  'Laughter',
  'Oboe',
  'Flute',
  'Hi-hat',
  'Cough',
  'Telephone',
  'Snare_drum',
  'Flute',
  'Bark',
  'Glockenspiel',
  'Chime',
  'Bass_drum',
  'Fart',
  'Cello',
  'Bus',
  'Squeak',
  'Scissors',
  'Harmonica',
  'Squeak',
  'Clarinet',
  'Scissors',
  'Clarinet',
  'Gong',
  'Writing',
  'Microwave_oven',
  'Burping_or_eructation',
  'Harmonica',
  'Cello',
  'Double_bass',
  'Double_bass',
  'Shatter',
  'Hi-hat',
  'Fireworks',
  'Hi-hat',
  'Fireworks',
  'Bark',
  'Tambourine',
  'Shatter',
  'Gong',
  'Harmonica',
  'Double_bass',
  '

In [18]:
model = load_model(model_path)

In [49]:
num_rows = 40
num_columns = 2584#216#2584#216#normalized_mfcc.shape[1]
num_channels = 1

def prepare(file_path:str):

    # using librosa to load the wav file
    y, sr = librosa.load(file_path) # ,duration=5

    # normalize the audio with librosa
    y = librosa.util.normalize(y)

    # Extract MFCC data
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

    # normalize the mfcc between -1 and 1
    #mfcc = librosa.util.normalize(mfcc)

    # reshape for neural network
    array = np.resize(mfcc, (num_rows, num_columns, num_channels))
    return array.reshape(1, array.shape[0], array.shape[1], array.shape[2])


# # compute a short-time Fourier transform (STFT)
# D = librosa.stft(normalized_y, n_fft=1024, hop_length=None)

# # convert to db
# D_db = librosa.amplitude_to_db(np.abs(D))

In [63]:
sample = metadata.sample()
wav_dir_of_sample = os.path.join(dataset_train_folder, sample['fname'].values[0])

# get prediction
predictions = model.predict([prepare(wav_dir_of_sample)])

index = np.argmax(predictions, axis=None, out=None)#np.argmax(predictions)
print(f"{sample['label'].values[0]} -> {labels[index]}")



#labels[np.argmax(predictions, axis=None, out=None)]

# print(f"{sample['label'].values[0]} -> {labels[index]}")
#print(f"selected sample: {sample['label']}\npredicted: {labels_npy[np.argmax(predictions)]}\ni:{np.argmax(predictions)}")
#sample

Saxophone -> Bus


In [ ]:
# from tensorflow.python.keras.saving import hdf5_format
# import h5py


# # Save model
# with h5py.File(model_path, mode='w') as f:
#     hdf5_format.save_model_to_hdf5(my_keras_model, f)
#     f.attrs['param1'] = param1
#     f.attrs['param2'] = param2

# # Load model
# with h5py.File(model_path, mode='r') as f:
#     param1 = f.attrs['param1']
#     param2 = f.attrs['param2']
#     my_keras_model = hdf5_format.load_model_from_hdf5(f)

In [ ]:
# import random
# import math
# import importlib

# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report, confusion_matrix
# from tensorflow.keras import backend as keras_backend
# from tensorflow.keras.models import Sequential, load_model
# from tensorflow.keras.layers import Dense, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, LeakyReLU
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.callbacks import ModelCheckpoint 
# from tensorflow.keras.regularizers import l2

# # Pre-processed MFCC coefficients
# X = np.load( os.path.join(data_npy_folder, "X-mfcc.npy" ) )
# y = np.load( os.path.join(data_npy_folder, "y-mfcc.npy" ) )

# # create train and test sets
# indexes = []
# total = len(metadata)
# indexes = list(range(0, total))

# # Randomize indexes
# random.shuffle(indexes)

# # Divide the indexes into Train and Test
# test_split_pct = 0#20
# split_offset = math.floor(test_split_pct * total / 100)

# # # Split the metadata
# test_split_idx = indexes[0:split_offset]
# #train_split_idx = indexes[split_offset:total]

# # Split the features with the same indexes
# X_test = np.take(X, test_split_idx, axis=0)
# y_test = np.take(y, test_split_idx, axis=0)

# # encode the labels to categorical
# le = LabelEncoder()
# y_test_encoded = to_categorical(le.fit_transform(y_test))

In [ ]:
# # create train and test sets
# indexes = []
# total = len(metadata)
# indexes = list(range(0, total))

# # Randomize indexes
# random.shuffle(indexes)

# # Divide the indexes into Train and Test
# test_split_pct = 0#20
# split_offset = math.floor(test_split_pct * total / 100)

# # # Split the metadata
# test_split_idx = indexes[0:split_offset]
# train_split_idx = indexes[split_offset:total]

# # Split the features with the same indexes
# X_test = np.take(X, test_split_idx, axis=0)
# y_test = np.take(y, test_split_idx, axis=0)
# X_train = np.take(X, train_split_idx, axis=0)
# y_train = np.take(y, train_split_idx, axis=0)


# # Also split metadata
# test_meta = metadata.iloc[test_split_idx]
# train_meta = metadata.iloc[train_split_idx]

# # Print status
# print("Test split: {} \t\t Train split: {}".format(len(test_meta), len(train_meta)))
# print("X test shape: {} \t X train shape: {}".format(X_test.shape, X_train.shape))
# print("y test shape: {} \t\t y train shape: {}".format(y_test.shape, y_train.shape))

In [ ]:

# # encode the labels to categorical
# le = LabelEncoder()
# y_test_encoded = to_categorical(le.fit_transform(y_test))
# y_train_encoded = to_categorical(le.fit_transform(y_train))

NameError: name 'y_test' is not defined

In [ ]:
# # Predict probabilities for test set
# y_probs = model.predict(X_test, verbose=0)

# # Get predicted labels
# yhat_probs = np.argmax(y_probs, axis=1)
# y_trues = np.argmax(y_test_encoded, axis=1)

# importlib.reload(helper)

# # Sets decimal precision (for printing output only)
# np.set_printoptions(precision=2)

# # Compute confusion matrix data
# cm = helper.confusion_matrix(y_trues, yhat_probs)

# helper.plot_confusion_matrix(
#     cm,
#     y,#y_test, 
#     normalized=False, 
#     title="Model Performance", 
#     cmap=plt.cm.Blues,
#     size=(12,12))

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.